# In-depth Analysis (Applying Machine Learning)

## Step 1) Read the Manual

Before we progress further, we display the information about the dataset that we obtained from the dataset manual, that is, from Kaggle and the UCI Machine Learning Repository.

From Kaggle, an overview of the variables:

There are 25 variables:

* ID: ID of each client
* LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
* SEX: Gender (1=male, 2=female)
* EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
* MARRIAGE: Marital status (1=married, 2=single, 3=others)
* AGE: Age in years
* PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
* PAY_2: Repayment status in August, 2005 (scale same as above)
* PAY_3: Repayment status in July, 2005 (scale same as above)
* PAY_4: Repayment status in June, 2005 (scale same as above)
* PAY_5: Repayment status in May, 2005 (scale same as above)
* PAY_6: Repayment status in April, 2005 (scale same as above)
* BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
* BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
* BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
* BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
* BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
* BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
* PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
* PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
* PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
* PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
* PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
* PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar) default.payment.next.month: Default payment (1=yes, 0=no)

And from UCI:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
* X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
* X2: Gender (1 = male; 2 = female). 
* X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
* X4: Marital status (1 = married; 2 = single; 3 = others). 
* X5: Age (year). 
* X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
* X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
* X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005. 

Potential issue: We'll want to group values 5 and 6 for Education into one value (looking at the Kaggle description) since they both stand for "unknown". And perhaps we'll want to include 4 in that grouping since it has the value of "others".

## Step 2) Review the Data Types

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import random
import sklearn

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
filename = 'UCI_Credit_Card.csv'

In [3]:
data = pd.read_csv(filename) #, index_col=0)
data = data.drop(columns='ID') # don't need ID column, because just the index + 1

In [4]:
pd.set_option('display.max_columns', 500)
data.sample(5).T

,26411,10651,29808,10034,11187
LIMIT_BAL,390000.0,500000.0,250000.0,270000.0,200000.0
SEX,2.0,1.0,1.0,1.0,2.0
EDUCATION,1.0,1.0,1.0,3.0,2.0
MARRIAGE,1.0,2.0,1.0,1.0,1.0
AGE,41.0,54.0,34.0,53.0,39.0
PAY_0,0.0,-2.0,0.0,0.0,-1.0
PAY_2,0.0,-2.0,0.0,0.0,2.0
PAY_3,2.0,-2.0,0.0,0.0,-1.0
PAY_4,0.0,-2.0,0.0,0.0,-1.0
PAY_5,0.0,-2.0,0.0,0.0,-1.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
LIMIT_BAL                     30000 non-null float64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null float64
BILL_AMT2                     30000 non-null float64
BILL_AMT3                     30000 non-null float64
BILL_AMT4                     30000 non-null float64
BILL_AMT5                     30000 non-null float64
BILL_AMT6                     30000 non-null float64
PAY_AMT1  

All columns in this dataset have a numeric type. They are either float-valued (continuous) or int-valued (discrete). Nothing seems to be off, so we may continue.

In [6]:
display(data.shape)

(30000, 24)

## Step 3) Fixing the Issues (Data Cleaning):

### Problem 1: Get rid of Bad Column Names


In [7]:
## Rename columns
data.rename(columns={'PAY_0': 'PAY_1', 'default.payment.next.month': 'default'}, inplace=True)

### Problem 2: Replace Negative Values with 0 in Pay_X columns

To deal with with values for the PAY_X columns, a sensible solution is to convert all non-positive values to 0. The dataset description says that a value of -1 means "pay duly" and positive values represent a payment delay by that number of months. Therefore, converting -1 and -2 values to 0, and having 0 represent "pay duly" is logical.

In [8]:
for i in range(1,7):
    data.loc[data["PAY_" + str(i)] < 0, "PAY_" + str(i)] = 0

### Problem 3: Get rid of Values of 0 for Marriage

A logical move is to group the 0 values with the "Other" values, coded as 3, so that is what we'll do:


In [9]:
data.loc[data["MARRIAGE"] == 0, 'MARRIAGE'] = 3

"Other" for marriage can possibly refer to divorced, widowed, seperated, etc.

### Problem 4: Get rid of 0 Values for Education

Currently coded as:
EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)

We see that 0 is not even in the dataset desciption, and we have 2 values for unknown. So a logical move is to convert the 0, 5 and 6 values to 4, which is what we'll do. "Other" can  refer to education less than high school or perhaps vocational training.

In [10]:
replace = (data["EDUCATION"] == 0) | (data["EDUCATION"] == 5) | (data["EDUCATION"] == 6) 
data.loc[replace,'EDUCATION'] = 4

## Step 4) Analysis

### Preprocessing

Now need to deal with SEX, EDUCATION, and MARRIAGE appropriately.

Previously Used

In [11]:
# replace_map = {'SEX': {1:"Male", 2:"Female"}, 'EDUCATION': {1: "Grad School", 2: "University", 3:"High School", 4:"Other"}, 'MARRIAGE': {1:"Married", 2:"Single", 3:"Other"}}
# data.replace(replace_map, inplace=True)

# data['default'] = data['default'].astype('category') 
# #Convert default variable from int64 to categorical variable

# data = pd.get_dummies(data, columns=['SEX', 'EDUCATION', 'MARRIAGE'], prefix=['SEX', 'EDUCATION', 'MARRIAGE'], drop_first=True)
# switch this to sklearn OneHotEncoder later

# # Not import because will just use X and y
# col_at_end = ['default']
# data = data[[column for column in data if column not in col_at_end] + [column for column in col_at_end if column in data]]
# ## Put default column at the end of the dataframe

# # Don't use Pandas categoricals
# data['PAY_1'] = data.PAY_1.astype('category')
# data['PAY_2'] = data.PAY_2.astype('category')
# data['PAY_3'] = data.PAY_3.astype('category')
# data['PAY_4'] = data.PAY_4.astype('category')
# data['PAY_5'] = data.PAY_5.astype('category')
# data['PAY_6'] = data.PAY_6.astype('category')


Let's examine the PAY_X columns now:

In [12]:
df = data[['PAY_6', 'PAY_5', 'BILL_AMT6', 'PAY_AMT6']]
df.columns = ['Repayment status in April', 'Repayment status in May', 'Amount of bill statement in April', 'Amount of previous payment in April']

In [13]:
df.loc[(df['Amount of bill statement in April'] < df['Amount of previous payment in April']) & (df['Repayment status in April'] < df['Repayment status in May'])].head()

,Repayment status in April,Repayment status in May,Amount of bill statement in April,Amount of previous payment in April
68,0,2,7319.0,13899.0
436,0,2,150.0,200.0
1875,0,2,107495.0,116880.0
2262,0,2,39819.0,223833.0
2272,0,2,244.0,856.0


Represent instances where our repayment status is **worse** and we've paid more than our bill.

### Steps Involved in Classification in Scikit-Learn

1. Preprocess Data
2. Create Train and Test Sets
3. Instantiate the model/estimator  
(Steps 1 and 3 can be combined in a Pipeline object)
4. Specify Hyperparameter Space
5. Instantiate GridSearchCV or RandomizedSearchCV objects
6. Fit CV object to the Training Set
7. Predict on the Test Set
8. Compute Scores for the Model

*Models*:
1. Logistic Regression (LR)
2. K-Nearest Neighbor (KNN)
3. Support Vector Machine (SVM)
4. Decision Trees (DT)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, scale, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
from scipy.stats import randint
import time

In [15]:
#### Step 1) Preprocess Data

# We will train our classifier with the following features:
# Numeric features to be scaled: LIMIT_BAL, AGE, PAY_X, BIL_AMTX, and PAY_AMTX
# Categorical features: SEX, EDUCATION, MARRIAGE

# We create the preprocessing pipelines for both numeric and categorical data
numeric_features = ['LIMIT_BAL', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 
                     'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 
                     'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
                   ]



data['PAY_1'] = data.PAY_1.astype('float64')
data['PAY_2'] = data.PAY_2.astype('float64')
data['PAY_3'] = data.PAY_3.astype('float64')
data['PAY_4'] = data.PAY_4.astype('float64')
data['PAY_5'] = data.PAY_5.astype('float64')
data['PAY_6'] = data.PAY_6.astype('float64')
data['AGE'] = data.AGE.astype('float64')

numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(categories='auto'))
])

In [16]:
# label_features = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
# label_transformer = Pipeline(steps=[
#     ('label', LabelEncoder())
# ])
# Use StratifiedShuffleSplit to get it to work

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        #,('lab', label_transformer, label_features)
    ])

In [18]:
#### Step 2) Split Data into Training and Test Sets

y = data['default']#.values
X = data.drop(['default'], axis=1)#.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

##### *1st Model: Logistic Regression*

In [ ]:
#### Step 3: Instantiate the Estimator

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
from sklearn.linear_model import LogisticRegression

lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegression(solver='liblinear'))])

#### Step 4: Specify the Hyperparameter Space

param_grid_lr = {
    
    'classifier__C': np.logspace(-5, 8, 15),
    'classifier__penalty': ['l1', 'l2']
}

#### Step 5: Instantiate the CV Object

lr_cv = GridSearchCV(lr, param_grid_lr, cv=5, iid=False)

#### Step 6: Fit on Training

t0 = time.time()
lr_cv.fit(X_train, y_train)
print("It takes ", time.time() - t0, " seconds for LR fitting")

#### Step 7: Predict on Test

y_pred_lr = lr_cv.predict(X_test)

#### Step 8: Scoring

##### Accuracy

print("Mean cross-validated score of the best estimator: %.3f" % lr_cv.best_score_)
# Latest accuracy is 0.82

print("Accuracy with LR on testing set is: %.3f" % lr_cv.score(X_test, y_test))
# Latest accuracy is 0.82

print("Accuracy with LR on training set is: %.3f" % lr_cv.score(X_train, y_train))
# Latest accuracy is 0.82

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
lr_cv2 = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegressionCV(solver='liblinear', cv=10, Cs=np.logspace(-5, 8, 15) ))])
lr_cv2.fit(X_train, y_train)

lr_cv2.score(X_test, y_test)

In [ ]:
lr_cv2.named_steps['classifier'].C_

###### Best parameters

In [ ]:
print("mean cross-validated score of the best_estimator: %.3f " % lr_cv.best_score_)
print("Parameter setting that gave the best results on the hold out data: C = %.3f" % lr_cv.best_params_.get('classifier__C'))
print("Parameter setting that gave the best results on the hold out data: penalty =", lr_cv.best_params_.get('classifier__penalty'))

In [ ]:
coefs = lr_cv.best_estimator_.named_steps['classifier'].coef_
coefs

In [ ]:
print("Total Number of Features:", coefs.size)
print(" Number of Selected Features:", np.count_nonzero(coefs))

In [ ]:
X_train.shape

In [ ]:
train_data = X_train.join(y_train)
test_data = X_test.join(y_test)

if ('prob_of_default' in train_data):
    train_data = train_data.drop(columns=['prob_of_default'])
if ('prob_of_default' in test_data):
    test_data = test_data.drop(columns=['prob_of_default'])

probs_train = lr_cv.predict_proba(X_train)[:,1] # probs = probability of default
probs_test = lr_cv.predict_proba(X_test)[:,1] # probs = probability of default

train_data['prob_of_default'] = probs_train
test_data['prob_of_default'] = probs_test

train_data['train/test'] = 'train'
test_data['train/test'] = 'test'

new_data = train_data.append(test_data).sort_index()

In [ ]:
train_ranked = new_data.loc[new_data['train/test'] == 'train'].sort_values(by='prob_of_default')
test_ranked = new_data.loc[new_data['train/test'] == 'test'].sort_values(by='prob_of_default')

These guys in the training set default with a low probability of default:

In [ ]:
train_ranked.loc[(train_ranked.prob_of_default < 0.5) & (train_ranked.default == 1)].head(1)

1. Individual 18967 defaults with an assigned default probability of about 1%. This seems to have happened because she had a very large bill statement in September, just before the bankruptcy point. One takeaway then is to look at all customers with large bills in September and see how many default. 

In [ ]:
train_ranked.loc[(train_ranked.PAY_1 == 0 ) & (train_ranked.BILL_AMT1 > 0)].default.mean()

It's not the case that customers with PAY_1 = 0 and BILL_AMT1 > 0 default more than the average. Actually, they default less than the average. What about if we also factor in PAY_AMT1?

In [ ]:
train_ranked.loc[(train_ranked.PAY_1 == 0 ) & (train_ranked.BILL_AMT1 > 0) & (train_ranked.PAY_AMT1 == 0)].default.mean()

Definately higher, but only about the mean now. Can't create rule based on this.

In [ ]:
# BILL_AMTX is related to PAY_AMTX-1, not PAY_AMTX
for i in range(6,1,-1):
    corr1 = new_data['BILL_AMT' + str(i)].corr(new_data['PAY_AMT' + str(i-1)])
    corr2 = new_data['BILL_AMT' + str(i)].corr(new_data['PAY_AMT' + str(i)])
    print(corr1 - corr2)

In [ ]:
train_ranked.corr()['prob_of_default'].sort_values(ascending=False)

In [ ]:
train_ranked.sort_values(by='BILL_AMT1', ascending=False).loc[:,['BILL_AMT1', 'PAY_AMT1', 'default', 'prob_of_default']]

**Classification Report**

In [ ]:
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

**ROC Curve**

In [ ]:
y_pred_prob = lr_cv.predict_proba(X_test)[:,1]
fpr, tpr, thresholds  = roc_curve(y_test, y_pred_prob)

In [ ]:
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

*Area under the Curve*

In [ ]:
roc_auc_score(y_test, y_pred_prob)

In [ ]:
cv_scores = cross_val_score(lr_cv, X, y, cv=5, scoring='roc_auc')
cv_scores

#### 2nd Model: KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#3
knn = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', KNeighborsClassifier())])
#4
param_grid_knn = {
    'classifier__n_neighbors': np.arange(1,20)
}

#5
knn_cv = RandomizedSearchCV(knn, param_grid_knn, cv=3, iid=False)

#6
t0 = time.time()
knn_cv.fit(X_train, y_train)
print("It takes ", time.time() - t0, " seconds for KNN fitting")
# takes 185 seconds with 5 folds, 1 to 20 neighbors,

#7
y_pred_knn = knn_cv.predict(X_test)

#8
print("Accuracy with KNN is: ", knn_cv.score(X_test, y_test))
# latest accuracy is 0.81

In [ ]:
print("mean cross-validated score of the best_estimator: %.3f " % knn_cv.best_score_)
print("Parameter setting that gave the best results on the hold out data: n_neighbors =", knn_cv.best_params_.get('classifier__n_neighbors'))

**Classification Report**

In [ ]:
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

**ROC Curve**

In [ ]:
y_pred_prob = knn_cv.predict_proba(X_test)[:,1]
fpr, tpr, thresholds  = roc_curve(y_test, y_pred_prob)

In [ ]:
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

*Area under the Curve*

In [ ]:
roc_auc_score(y_test, y_pred_prob)

In [ ]:
cv_scores = cross_val_score(knn_cv, X, y, cv=5, scoring='roc_auc')
cv_scores

#### 3nd Model: Support Vector Machine

In [ ]:
from sklearn.svm import SVC
#3
svm = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', SVC())])
#4
param_grid_svm = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__gamma': [1, 0.1, 0.01]
}

#5
svm_cv = RandomizedSearchCV(svm, param_grid_svm, cv=3, iid=False)

#6
t0 = time.time()
svm_cv.fit(X_train, y_train)
print("It takes ", time.time() - t0, " seconds for SVM fitting")
# takes 272 seconds with 3 folds

#7
y_pred_svm = svm_cv.predict(X_test)

#8
print("Mean cross-validated score of the best estimator: %.3f" % svm_cv.best_score_)
print("Accuracy with LR on testing set is: %.3f" % svm_cv.score(X_test, y_test))
print("Accuracy with LR on training set is: %.3f" % svm_cv.score(X_train, y_train))

y_pred_prob = svm_cv.predict_proba(X_test)[:,1]
print("ROC AUC score is: %.3f" % roc_auc_score(y_test, y_pred_prob))

#### 4th Model: Decision Tree

In [ ]:
# Import necessary modules 
from sklearn.tree import DecisionTreeClassifier

#3
dt = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', DecisionTreeClassifier())])
#4
# Setup the parameters and distributions to sample from: param_dist
param_grid_dt = {'classifier__max_depth': [3, None],
                'classifier__max_features': randint(1, 9),
                'classifier__min_samples_leaf': randint(1, 9),
                'classifier__criterion': ["gini", "entropy"]}

#5
dt_cv = RandomizedSearchCV(dt, param_grid_dt, cv=10, iid=False)

#6
t0 = time.time()
dt_cv.fit(X_train, y_train)
print("It takes ", time.time() - t0, " seconds for DT fitting")
# takes 11 seconds with 10 folds

#7
y_pred_dt = dt_cv.predict(X_test)

#8
print("Mean cross-validated score of the best estimator: %.3f" % dt_cv.best_score_)
print("Accuracy with LR on testing set is: %.3f" % dt_cv.score(X_test, y_test))
print("Accuracy with LR on training set is: %.3f" % dt_cv.score(X_train, y_train))

y_pred_prob = dt_cv.predict_proba(X_test)[:,1]
print("ROC AUC score is: %.3f" % roc_auc_score(y_test, y_pred_prob))

#### 5th Model: RandomForest

In [ ]:
%run import_data.py

In [ ]:
%run -i std_preprocessing_and_splitting.py

In [19]:
# Import necessary modules 

from sklearn.ensemble import RandomForestClassifier

#3
rf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', RandomForestClassifier())])
#4
# Setup the parameters and distributions to sample from: param_dist
param_grid_rf = {'classifier__n_estimators': [1, 10, 50, 100]}

#5
rf_cv = GridSearchCV(rf, param_grid_rf, cv=10, iid=False)

#6
t0 = time.time()
rf_cv.fit(X_train, y_train)
print("It takes ", time.time() - t0, " seconds for Random Forest fitting")

#7
y_pred_rf = rf_cv.predict(X_test)

#8
print("Mean cross-validated score of the best estimator: %.3f" % rf_cv.best_score_)
print("Accuracy with Random Forest on testing set is: %.3f" % rf_cv.score(X_test, y_test))
print("Accuracy with Random Forest on training set is: %.3f" % rf_cv.score(X_train, y_train))

y_pred_prob = rf_cv.predict_proba(X_test)[:,1]
print("ROC AUC score is: %.3f" % roc_auc_score(y_test, y_pred_prob))

It takes  76.9420096874237  seconds for Random Forest fitting
Mean cross-validated score of the best estimator: 0.813
Accuracy with LR on testing set is: 0.820
Accuracy with LR on training set is: 0.999
ROC AUC score is: 0.781
